In [ ]:
# model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as tg
import torch_scatter
import e3nn
from e3nn.nn.models.v2103.gate_points_networks import SimpleNetwork
from typing import Dict, Union

# crystal structure data
from pymatgen.core.periodic_table import Element
from ase import Atoms
from ase.neighborlist import neighbor_list

# data pre-processing and visualization
import numpy as np
import matplotlib.pyplot as plt

# utilities
import time
from tqdm import tqdm
from utils import load_data, train_valid_test_split, plot_example, train, plot_predictions, bar_format, colors

default_dtype = torch.float64
torch.set_default_dtype(default_dtype)

In [ ]:
# reload a module without kernel restart
from importlib import reload
import sys
reload(sys.modules['utils'])
from utils import load_data, train_valid_test_split, train, plot_predictions, bar_format, colors

In [ ]:
def build_data(entry, type_encoding, type_onehot, r_max=5.):
    symbols = [specie.symbol for specie in entry.structure.species]
    positions = torch.from_numpy(entry.structure.cart_coords.copy())
    cell = torch.from_numpy(entry.structure.lattice.matrix.copy())
    crystal = Atoms(symbols=symbols, positions=positions, cell=cell, pbc=True)

    # edge_src and edge_dst are the indices of the central and neighboring atom, respectively
    # edge_shift indicates whether the neighbors are in different images / copies of the unit cell
    edge_src, edge_dst, edge_shift = neighbor_list("ijS", a=crystal, cutoff=r_max, self_interaction=True)
    
    # calculate edge lengths
    batch = positions.new_zeros(positions.shape[0], dtype=torch.long)
    edge_batch = batch[torch.from_numpy(edge_src)]
    edge_vec = (positions[torch.from_numpy(edge_dst)]
                - positions[torch.from_numpy(edge_src)]
                + torch.einsum('ni,nij->j', torch.tensor(edge_shift, dtype=default_dtype),
                               cell.unsqueeze(0)[edge_batch]))
    edge_len = np.around(np.sqrt((edge_vec**2).sum(dim=1).numpy()), decimals=2)
    
    data = tg.data.Data(
        pos=positions,
        lattice=cell.unsqueeze(0),
        symbol=symbols,
        x=type_onehot[[type_encoding[specie] for specie in symbols]],
        edge_index=torch.stack([torch.LongTensor(edge_src), torch.LongTensor(edge_dst)], dim=0),
        edge_shift=torch.tensor(edge_shift, dtype=default_dtype),
        edge_len=edge_len,
        phdos=torch.from_numpy(entry.phdos).unsqueeze(0)
    )
    
    return data


class SimplePeriodicNetwork(SimpleNetwork):
    def __init__(self, in_dim, em_dim, **kwargs):
        """The keyword `pool_nodes` is used by SimpleNetwork to determine
        whether we sum over all atom contributions per example. In this example,
        we want use a mean operation instead, so we will override this behavior.
        """
        
        self.pool = False
        if kwargs['pool_nodes'] == True:
            kwargs['pool_nodes'] = False
            kwargs['num_nodes'] = 1.
            self.pool = True
        super().__init__(**kwargs)

        # further embed the mass-weighted one-hot encoding
        self.fc1 = nn.Linear(in_dim, em_dim)

    # Overwriting preprocess method of SimpleNetwork to adapt for periodic boundary data
    def preprocess(self, data: Union[tg.data.Data, Dict[str, torch.Tensor]]) -> torch.Tensor:
        if 'batch' in data:
            batch = data['batch']
        else:
            batch = data['pos'].new_zeros(data['pos'].shape[0], dtype=torch.long)

        edge_src = data['edge_index'][0]  # Edge source
        edge_dst = data['edge_index'][1]  # Edge destination

        # We need to compute this in the computation graph to backprop to positions
        # We are computing the relative distances + unit cell shifts from periodic boundaries
        edge_batch = batch[edge_src]
        edge_vec = (data['pos'][edge_dst]
                    - data['pos'][edge_src]
                    + torch.einsum('ni,nij->j', data['edge_shift'], data['lattice'][edge_batch]))

        return batch, data['x'], edge_src, edge_dst, edge_vec

    def forward(self, data: Union[tg.data.Data, Dict[str, torch.Tensor]]) -> torch.Tensor:
        # if pool_nodes was set to True, use scatter_mean to aggregate
        data.x = F.relu(self.fc1(data.x))
        output = super().forward(data)
        
        if self.pool == True:
            output = torch_scatter.scatter_mean(output, data.batch, dim=0)  # Take mean over atoms per example
            # output = torch.sigmoid(output)
            output = F.relu(output)
            maxima, _ = torch.max(output, dim=1)
            output = output.div(maxima.unsqueeze(1))
            return output
        
        else:
            # output = torch.sigmoid(output)
            output = F.relu(output)
            maxima, _ = torch.max(output, dim=1)
            output = output.div(maxima.unsqueeze(1))
            return output

In [ ]:
# load data
df, species = load_data('data.csv')
df.head()

In [ ]:
# train/valid/test split
idx_train, idx_valid, idx_test = train_valid_test_split(df, species, seed=12, plot=True)

In [ ]:
# one-hot encoding atomic mass
type_encoding = {}
specie_am = []
for Z in tqdm(range(1, 119), bar_format=bar_format):
    specie = Element.from_Z(Z)
    type_encoding[specie.symbol] = Z
    specie_am.append(specie.atomic_mass)

type_onehot = torch.diag(torch.tensor(specie_am))

In [ ]:
# build data
r_max = 5 # cutoff radius
df['data'] = df.progress_apply(lambda x: build_data(x, type_encoding, type_onehot, r_max), axis=1)

In [ ]:
plot_example(df, i=12, label_edges=True)

In [ ]:
batch_size = 1
dataloader_train = tg.data.DataLoader(df.iloc[idx_train]['data'].values, batch_size=batch_size, shuffle=True)
dataloader_valid = tg.data.DataLoader(df.iloc[idx_valid]['data'].values, batch_size=batch_size)
dataloader_test = tg.data.DataLoader(df.iloc[idx_test]['data'].values, batch_size=batch_size)

In [ ]:
# calculate average number of neighbors
def get_neighbors(df, idx):
    n = []
    for entry in df.iloc[idx].itertuples():
        N = entry.data.pos.shape[0]
        for i in range(N):
            n.append(len((entry.data.edge_index[0] == i).nonzero()))
    return np.array(n)

n_train = get_neighbors(df, idx_train)
n_valid = get_neighbors(df, idx_valid)
n_test = get_neighbors(df, idx_test)

fig, ax = plt.subplots(1,1, figsize=(5,4))
b = 0.
bins = 50
for (d, c), n in zip(colors.items(), [n_train, n_valid, n_test]):
    color = [int(c.lstrip('#')[i:i+2], 16)/255. for i in (0,2,4)]
    y, bins, _, = ax.hist(n, bins=bins, fc=color+[0.7], ec=color, bottom=b, label=d)
    b += y
ax.set_xlabel('number of neighbors')
ax.set_ylabel('number of examples')
ax.legend(frameon=False)

print('average number of neighbors (train/valid/test):', n_train.mean(), '/', n_valid.mean(), '/', n_test.mean())

In [ ]:
out_dim = len(df.iloc[0]['phfreq'])
em_dim = 64
model = SimplePeriodicNetwork(
    in_dim=118,
    em_dim=em_dim,
    irreps_in=str(em_dim)+"x0e",    # one-hot scalars (L=0 and even parity) on each atom to represent atom type
    irreps_out=str(out_dim)+"x0e",  # out_dim scalars (L=0 and even parity) to output
    max_radius=r_max,               # cutoff radius for convolution
    num_neighbors=n_train.mean(),   # scaling factor based on the typical number of neighbors
    pool_nodes=True,
)

print(model)

In [ ]:
opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.96)

loss_fn = torch.nn.MSELoss()
loss_fn_mae = torch.nn.L1Loss()

In [ ]:
device = "cuda:7" if torch.cuda.is_available() else "cpu"
print('torch device:' , device)

run_name = 'models/' + time.strftime("%y%m%d", time.localtime())
print(run_name)

In [ ]:
for results in train(model, opt, dataloader_train, dataloader_valid, loss_fn, loss_fn_mae, max_iter=64,
                     scheduler=scheduler, device=device):
    with open(run_name + '_test.torch', 'wb') as f:
        torch.save(results, f)

In [ ]:
saved = torch.load(run_name + '_test.torch')
steps = [d['step'] + 1 for d in saved['dynamics']]
valid = [d['valid']['loss'] for d in saved['dynamics']]
train = [d['train']['loss'] for d in saved['dynamics']]

fig, ax = plt.subplots(figsize=(6,5))
ax.plot(steps, train, 'o-', label="train", color=colors['train'])
ax.plot(steps, valid, 'o-', label="valid", color=colors['valid'])
ax.set_xlabel('epochs')
ax.set_ylabel('loss')
ax.legend(frameon=False);

In [ ]:
# predict on all data
dataloader = tg.data.DataLoader(df['data'].values, batch_size=64)
df['mse'] = 0.
df['phdos_pred'] = np.empty((len(df), 0)).tolist()

model.eval()
with torch.no_grad():
    i0 = 0
    for i, d in tqdm(enumerate(dataloader), total=len(dataloader), bar_format=bar_format):
        d.to(device)
        output = model(d)
        loss = F.mse_loss(output, d.phdos, reduction='none').mean(dim=-1).cpu().numpy()
        df.loc[i0:i0 + len(d.phdos) - 1, 'phdos_pred'] = [[k] for k in output.cpu().numpy()]
        df.loc[i0:i0 + len(d.phdos) - 1, 'mse'] = loss
        i0 += len(d.phdos)
        
df['phdos_pred'] = df['phdos_pred'].map(lambda x: x[0])

In [ ]:
plot_predictions(df, idx_train, 'Training')

In [ ]:
plot_predictions(df, idx_valid, 'Validation')

In [ ]:
plot_predictions(df, idx_test, 'Testing')